In [4]:
# importing required libraries
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [5]:
df = pd.read_excel('Mumbai_Neighborhoods.xlsx')
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].astype(float)
df.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270


In [6]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 19.0759899, 72.8773928.


In [7]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [9]:
# storing Foursquare credentials
CLIENT_ID = 'YOETKBCTDQQCNBQ4F523LHHFWKAWNS2M1FO2HXNJFDVSCZXX' # your Foursquare ID
CLIENT_SECRET = 'MCFQMJ3ELY1ATDUXEXO5TFPKOW3TQJZFDP1XGRVMIJ44F1GC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOETKBCTDQQCNBQ4F523LHHFWKAWNS2M1FO2HXNJFDVSCZXX
CLIENT_SECRET:MCFQMJ3ELY1ATDUXEXO5TFPKOW3TQJZFDP1XGRVMIJ44F1GC


In [11]:
# creating url for FourSquare API Call
LIMIT = 100 
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    area_latitude, 
    area_longitude, 
    radius, 
    LIMIT)

url 

'https://api.foursquare.com/v2/venues/explore?&client_id=YOETKBCTDQQCNBQ4F523LHHFWKAWNS2M1FO2HXNJFDVSCZXX&client_secret=MCFQMJ3ELY1ATDUXEXO5TFPKOW3TQJZFDP1XGRVMIJ44F1GC&v=20180605&ll=19.1293,72.8434&radius=1000&limit=100'

In [12]:
# calling Foursquare API for all the areas in Mumbai
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
mumbai_venues = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Amboli
Chakala, Andheri
D.N. Nagar
Four Bungalows
Lokhandwala
Marol
Sahar
Seven Bungalows
Versova
Mira Road
Bhayandar
Uttan
Bandstand Promenade
Kherwadi
Pali Hill
I.C. Colony
Gorai
Dahisa
Aarey Milk Colony
Bangur Nagar
Jogeshwari West
Juhu
Charkop
Poisar
Mahavir Nagar
Thakur village
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Kalina
Naigaon
Nalasopara
Virar
Irla
Vile Parle
Bhandup
Amrut Nagar
Asalfa
Pant Nagar
Kanjurmarg
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Indian Institute of Technology Bombay campus
Vidyavihar
Vikhroli
Chembur
Deonar
Mankhurd
Mahul
Agripada
Altamount Road
Bhuleshwar
Breach Candy
Carmichael Road
Cavel
Churchgate
Cotton Green
Cuffe Parade
Cumbala Hill
Currey Road
Dhobitalao
Dongri
Kala Ghoda
Kemps Corner
Lower Parel
Mahalaxmi
Mahim
Malabar Hill
Marine Drive
Marine Lines
Mumbai Central
Nariman Point
Prabhadevi
Sion
Walkeshwar
Worli
C.G.S. colony
Dagdi Chawl
Navy Nagar
Hindu colony
Ballard Estate
Chira Bazaar
Fanas Wadi
Chor Bazaar
Matunga
Parel
Gowalia Tank
D

In [14]:
print(mumbai_venues.shape)
mumbai_venues.head()

(1319, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.8434,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
1,Amboli,19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
2,Amboli,19.1293,72.8434,Subway,19.127860,72.844461,Sandwich Place
3,Amboli,19.1293,72.8434,Cafe Coffee Day,19.127748,72.844663,Coffee Shop
4,Amboli,19.1293,72.8434,Bhardawadi Ground,19.126143,72.843548,Park


In [18]:
dataframe = mumbai_venues[mumbai_venues['Venue Category'].str.contains("Restaurant")].reset_index(drop = True)
dataframe.head()

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.129300,72.843400,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
1,Amboli,19.129300,72.843400,Cafe Arfa,19.128930,72.847140,Indian Restaurant
2,Amboli,19.129300,72.843400,Nukkad Food Bistro,19.126058,72.846618,Fast Food Restaurant
3,"Chakala, Andheri",19.111388,72.860833,Faaso's,19.113938,72.862330,Fast Food Restaurant
4,"Chakala, Andheri",19.111388,72.860833,Hit & Run,19.107787,72.863333,Falafel Restaurant


In [25]:
# one hot encoding
mumbai_onehot = pd.get_dummies(dataframe[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Area'] = dataframe['Area'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

#creating mumbai_grouped for clustering
mumbai_grouped = mumbai_onehot.groupby('Area').mean().reset_index()
mumbai_grouped

,Area,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Goan Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Maharashtrian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mughlai Restaurant,New American Restaurant,North Indian Restaurant,Parsi Restaurant,Punjabi Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Tex-Mex Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant
0,Agripada,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Altamount Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Amboli,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.0000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Amrut Nagar,0.058824,0.058824,0.117647,0.000000,0.000000,0.058824,0.0000,0.00,0.058824,0.117647,0.000000,0.000000,0.000000,0.0000,0.294118,0.058824,0.000000,0.000000,0.058824,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bandstand Promenade,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.0000,0.250000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bhandup,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
6,Bhuleshwar,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.090909,0.000000,0.000000,0.000000,0.0000,0.727273,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000
7,Breach Candy,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.00,0.000000,0.166667,0.166667,0.000000,0.000000,0.0000,0.166667,0.166667,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,C.G.S. colony,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Carmichael Road,0.000000,0.000000,0.000000,0.142857,0.000000,0.428571,0.0000,0.00,0.000000,0.285714,0.000000,0.000000,0.000000,0.0000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
# set number of clusters
kclusters = 10
mumbai_grouped_clustering = mumbai_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# add clustering labels
mumbai_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = df

# merge mumbai_grouped with mumbai_data to add latitude/longitude for each area
mumbai_merged = mumbai_merged.join(mumbai_grouped.set_index('Area'), on='Area')

mumbai_merged.head() # check the last columns!

,Area,Location,Latitude,Longitude,Cluster Labels,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Goan Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Maharashtrian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mughlai Restaurant,New American Restaurant,North Indian Restaurant,Parsi Restaurant,Punjabi Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Tex-Mex Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,8.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,4.0,0.0,0.0,0.111111,0.0,0.0,0.000000,0.0,0.0,0.111111,0.222222,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.333333,0.111111,0.0,0.0,0.0,0.0
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,3.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,6.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,4.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.0


In [33]:
mumbai_merged_new = mumbai_merged.dropna()
mumbai_merged_new

,Area,Location,Latitude,Longitude,Cluster Labels,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Goan Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Maharashtrian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mughlai Restaurant,New American Restaurant,North Indian Restaurant,Parsi Restaurant,Punjabi Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Tex-Mex Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.0000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,4.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0000,0.00,0.111111,0.222222,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.111111,0.333333,0.111111,0.000000,0.000000,0.000000,0.000000
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Marol,"Andheri,Western Suburbs",19.119219,72.882743,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.750000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018,8.0,0.000000,0.000000,0.090909,0.000000,0.000000,0.272727,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.272727,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.090909,0.181818,0.090909,0.000000,0.000000,0.000000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333
11,Uttan,"Mira-Bhayandar,Western Suburbs",19.280000,72.785000,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,Bandstand Promenade,"Bandra,Western Suburbs",19.042718,72.819132,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.0000,0.250000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged_new['Latitude'], mumbai_merged_new['Longitude'], mumbai_merged_new['Area'], mumbai_merged_new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters